<a href="https://colab.research.google.com/github/yukinaga/minnano_cs/blob/main/section_6/01_lifegame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ライフゲーム
ライフゲームは、イギリスの数学者Conwayが考案したセル・オートマトンの一種です。  
ライフゲームのセルには、死んでいるセル（0）と生きているセル（1）の2種類があります。これらは、以下のルールにより次の時刻における生死が決まります。  

* 死んでいるセル（0）の周囲に3つの生きているセル（1）があれば次の時刻では生きているセルになる
* 生きているセルの周囲に2つか3つの生きているセルがあれば次の時刻でも生きているセルのまま
* これ以外の場合は、死んだセルになる

Pythonを使い、ライフゲームを実装しましょう。

## ◎ライフゲーム実装の準備
ライフゲームの実装のために、数値演算ライブラリNumPyの2次元配列について学びましょう。  
以下のセルでは、2次元配列`a`を作成して表示します。  


In [ ]:
import numpy as np  # NumPyの導入、以降はnpという名前で使用

# 2次元配列
a = np.array([[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12],
              [13, 14, 15, 16]])
print(a)

同様に2次元配列`b`を作成します。  
配列同士は、形状が同じなのであれば足し合わせることができます。  
以下のセルでは、演算子`+`を使って`a`と`b`を足し合わせています。  

In [ ]:
b = np.array([[16, 15, 14, 13],
              [12, 11, 10, 9],
              [8, 7, 6, 5],
              [4, 3, 2, 1]])
print(a + b)  # aとbの各要素を足し合わせる

今回は、ライフゲームの実装のためにNumPyの`roll`関数を使います。  
`roll`関数を使えば、2次元配列の要素を縦もしくは横にずらすことができます。  
`roll`関数は、以下のようして使うことができます。  

```ずらされた配列 = np.roll(元の配列, (ずれの値, ずれの値), axis=(ずれの方向、ずれの方向))```  

例えば`np.roll(a, (1, 0), axis=(0, 1))`は、axisが0の方向（縦）に1ずらし、axisが1の方向（横）に0ずらすことを意味します。  
以下のセルのコードでは、様々なずれの値を設定した上で`roll`関数を実行します。



In [ ]:
print(np.roll(a, (1, 0), axis=(0, 1)))  # 0の方向に1、1の方向に0ずらす

print()

print(np.roll(a, (-1, 0), axis=(0, 1)))  # 0の方向に-1、1の方向に0ずらす

print()

print(np.roll(a, (0, 1), axis=(0, 1)))  # 0の方向に0、1の方向に1ずらす

print()

print(np.roll(a, (0, -1), axis=(0, 1)))  # 0の方向に0、1の方向に-1ずらす

NumPyの2次元配列と`roll`関数を使って、ライフゲームの実装にトライしましょう。

## @ 演習

以下のライフゲームの条件に基づき、2次元配列の各要素を更新する関数を記述しましょう。  
* 死んでいるセル（0）の周囲に3つの生きているセル（1）があれば次の時刻では生きているセルになる
* 生きているセルの周囲に2つか3つの生きているセルがあれば次の時刻でも生きているセルのまま
* これ以外の場合は、死んだセルになる

以下の`update_states`関数は、各要素が0もしくは1の2次元配列`states`を更新する関数です。  
コードに追記を行い、正しく状態`states`が更新されるようにしましょう。  


In [ ]:
def update_states(states):  # states: 各要素が0もしくは1の状態を表す2次元配列
    total = (  # 周囲の生きているセル（1）の数を数える
        np.roll(states, (-1, 0), axis=(0, 1)) + np.roll(states, (1, 0), axis=(0, 1))
        +   # ←こちらにコードを追記
        + np.roll(states, (-1, -1), axis=(0, 1)) + np.roll(states, (-1, 1), axis=(0, 1))
        +   # ←こちらにコードを追記
    )
    states = np.where(
        ((states==0) & (total==3))  # 死んでいるセル（0）の周囲に3つの生きているセル（1）
        | ((states==1) & ((total==2) | (total==3))),  # 生きているセル（1）の周囲に2つか3つの生きているセル（1）
        1, 0  # 生（1）か死（0）
        )
    return states

上記にコードを追記後、以下のセルを実行すると`update_states`関数を使ってライフゲームの処理が行われ、結果が表示されます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc

# @markdown 領域の高さと幅を入力してください。
height =  50# @param {type:"number"}
width =  50# @param {type:"number"}

# @markdown タイムステップを入力してください。
steps = 200# @param {type:"number"}

# @markdown GIFアニメを保存しますか？
save_gif = False #@param {type:"boolean"}

# @markdown MP4形式の動画を保存しますか？
save_mp4 = False #@param {type:"boolean"}

# 結果表示のための設定
figure = plt.figure()
plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
plt.tick_params(bottom=False, left=False, right=False, top=False)

# 初期値をランダムに設定
states = np.random.randint(0, 2, (height, width))

# 画像を格納するリスト
images = []
image = plt.imshow(states.tolist(), cmap="gray_r")
images.append([image])

# セルの更新
for t in range(steps):
    states = update_states(states)
    image = plt.imshow(states.tolist(), cmap="gray_r")
    images.append([image])

# アニメーションの表示
anim = animation.ArtistAnimation(figure, images, interval=100, blit=True)
rc("animation", html="jshtml")
if save_gif:
    anim.save("lifegame.gif", writer="pillow", fps=10)
if save_mp4:
    anim.save("lifegame.mp4", writer="ffmpeg")
plt.close()  # 通常のグラフを非表示に
anim

結果が表示されたら、再生ボタンをクリックしてライフゲームの過程を確認しましょう。  
なお、上記のセルのコードを確認したい方は、表示→コードを表示/非表示で表示することができます。

## @解答例

In [ ]:
def update_states(states):  # 各要素が0もしくは1の状態を表す2次元配列
    total = (  # 周囲の生きているセル（1）の数を数える
        np.roll(states, (-1, 0), axis=(0, 1)) + np.roll(states, (1, 0), axis=(0, 1))
        + np.roll(states, (0, -1), axis=(0, 1)) + np.roll(states, (0, 1), axis=(0, 1))  # ←こちらにコードを追記
        + np.roll(states, (-1, -1), axis=(0, 1)) + np.roll(states, (-1, 1), axis=(0, 1))
        + np.roll(states, (1, -1), axis=(0, 1)) + np.roll(states, (1, 1), axis=(0, 1))  # ←こちらにコードを追記
    )
    states = np.where(
        ((states==0) & (total==3))  # 死んでいるセル（0）の周囲に3つの生きているセル（1）
        | ((states==1) & ((total==2) | (total==3))),  # 生きているセル（1）の周囲に2つか3つの生きているセル（1）
        1, 0  # 生（1）か死（0）
        )
    return states